Задача. Сделать пайплайн обработки файла cars.csv.

Необходимо посчитать по каждому производителю (поле manufacturer_name):

кол-во объявлений
средний год выпуска автомобилей
минимальную цену
максимальную цену
Выгрузить результат в output.csv.

In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as t




def main():
    # 1 create spark session
    spark = SparkSession.builder.getOrCreate()
    
    # EXTRACT
    # read data
    df = spark.read.format("csv").option("header", "true").load("/home/jovyan/work/data/cars.csv")
    # check read data
    # df.show(5)
    
    #TRANSFORM
    
    output = (
        df
        # считаем по каждому производителю
        .groupBy("manufacturer_name")
        .agg(
            # первая метрика - количество авто
            F.count("manufacturer_name").alias ("count"),
            # вторая метрика  - средний год выпуска автомобилей
            F.round(F.avg("year_produced")).cast(t.IntegerType()).alias("avg_year"),
            # третья метрика - минимальная цена
            F.min(F.col("price_usd").cast(t.FloatType())).alias("min_price"),
            # четвертая метрика - максимальная цена
            F.max(F.col("price_usd").cast(t.FloatType())).alias("max_price")
        )
    
    )
    
    output.show()
    
    # LOAD
    
    output.coalesce(5).write.mode("overwrite").format("json").save("data/cars_result1.json")

    

    
main()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/06 16:51:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


AnalysisException: Path does not exist: file:/home/jovyan/work/data/cars.csv